| Name (Last, First) | Student ID | Section contributed  | Section edited      | Other contributions   |
|--------------------|------------|----------------------|---------------------|-----------------------|
| Hawlader, Antanila | 301332035  | Researched the codes | finding datasets    | provided functions    |
| Long, Jiang        | 200099436  | Inputted the codes   | edited all sections | inputted functions    |
| Savkovic, Sava     | 301397121  | Research codes       | researched reviews  | reviewed all sections |

In [85]:
import csv
import itertools
import math
import nltk

max_documents = 128000 # 'None' means no limit. Tokenizing all 485,212 reviews will take a LONG TIME

# The dataset CSV is from https://www.kaggle.com/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset
# We truncate the CSV so it's less than 100 MB (GitHub's file size limit)
# The truncated CSV contains 485,212 movie reviews from Rotten Tomatoes, each labeled 'Fresh' (positive) or 'Rotten' (negative)
file_path = 'rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_critic_reviews.csv'

# Open the file again for reading
f = open(file_path, encoding='UTF8')
csv_reader = csv.DictReader(f)

documents = []
documents_text = ''

for line in itertools.islice(csv_reader, 0, max_documents):
    # For some reason '10,000' shows up as an "informative feature", let's ignore comments with it
    if "10,000" in line['review_content']: 
        continue
    if(line['review_type']):
        sentiment = False
        if line['review_type'] == 'Fresh':
            sentiment = 'pos'
        if line['review_type'] == 'Rotten':
            sentiment = 'neg'
        if(sentiment):
            tokens = nltk.word_tokenize(line['review_content'])
            documents_text = documents_text + line['review_content']
            documents.append((tokens, sentiment))

documents_tokens = nltk.word_tokenize(documents_text)
print(f"Using {len(documents)} documents.")

Using 127959 documents.


In [86]:
max_word_features = 2000

all_words = nltk.FreqDist(w.lower() for w in documents_tokens)
word_features = list(all_words)[:max_word_features]

In [87]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(d), c) for (d,c) in documents]

In [ ]:
train_set_ratio = 0.9 # 0.9 means 90% of the data is for training, 10% is for testing

train_set_size = math.floor(len(documents) * train_set_ratio)

train_set, test_set = featuresets[:train_set_size], featuresets[train_set_size:]

classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
# Test the classifier
print(nltk.classify.accuracy(classifier, test_set))

# Show the most important features as interpreted by Naive Bayes
classifier.show_most_informative_features(5)

## References

Paul, Sayak. (2021, May 17) Python sentiment analysis tutorial. DataCamp Community. (n.d.). Retrieved March 10, 2022, from https://www.datacamp.com/community/tutorials/simplifying-sentiment-analysis-python

Leone, S. (2020, November 4). Rotten tomatoes movies and critic Reviews Dataset. Kaggle. Retrieved March 10, 2022, from https://www.kaggle.com/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset 